In [53]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np

In [54]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [55]:
df = pd.read_csv("/content/drive/MyDrive/프로젝트랩/Main_Dataset_std4.csv")
df_main = df.drop(columns = ['Unnamed: 0', 'TimeStamp', 'SATURATOR1_ML_SUPPLY_F_PV.Value', 'SATURATOR2_ML_SUPPLY_F_PV.Value'])
df_main = np.log(1+df_main)
df_main.insert(0, 'TimeStamp', df['TimeStamp'])
df_main

,TimeStamp,Pol,IS,Moisture,Ash,Filterability,Color,Alcohol_Floc,MELTER.MELTER_CONTROL_BX.Value,MELTER_ML.TEMP_PV_Value,...,SATURATOR2_PH_PV.Value,SATURATOR2_ST_TEMP.Value,SATURATOR2_CALCIUM_F.Value,SATURATOR3_PH_PV.Value,SATURATOR3_ST_TEMP.Value,SATURATOR1_RATE_SV.Value,SATURATOR_ML_SUPPLY_F_PV.Value,Saturator1_exp_CaO,Saturator2_exp_CaO,Saturator3_exp_CaO
0,2018-12-01 09:00:00,4.60406,0.268499,0.313423,0.244514,4.180675,7.487902,4.529368,4.189491,4.365853,...,2.301630,4.330595,6.640910,2.138443,4.295150,0.470004,4.445359,6.428105,6.390241,5.863631
1,2018-12-01 09:15:00,4.60406,0.268499,0.313423,0.244514,4.180675,7.487902,4.529368,4.202339,4.352696,...,2.286698,4.325911,6.616174,2.126592,4.293593,0.470004,4.399548,6.428105,6.390241,5.863631
2,2018-12-01 09:30:00,4.60406,0.268499,0.313423,0.244514,4.180675,7.487902,4.529368,4.198871,4.384653,...,2.277748,4.323966,6.568732,2.112202,4.291577,0.470004,4.391931,6.428105,6.390241,5.863631
3,2018-12-01 09:45:00,4.60406,0.268499,0.313423,0.244514,4.180675,7.487902,4.529368,4.181764,4.346083,...,2.284456,4.324646,6.587623,2.126024,4.291968,0.470004,4.397271,6.428105,6.390241,5.863631
4,2018-12-01 10:00:00,4.60406,0.268499,0.313423,0.244514,4.180675,7.487902,4.529368,4.180841,4.347690,...,2.270971,4.325379,6.521772,2.113509,4.291200,0.470004,4.340785,6.428105,6.390241,5.863631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75922,2022-04-30 07:45:00,4.60477,0.235862,0.116894,0.150143,4.056989,8.421783,4.334673,4.223518,4.366376,...,2.243356,4.290315,6.720830,2.188527,4.268016,0.598837,4.165130,6.308098,6.240276,5.686975
75923,2022-04-30 08:00:00,4.60477,0.235862,0.116894,0.150143,4.056989,8.421783,4.334673,4.231497,4.348620,...,2.335198,4.289011,6.557644,2.197916,4.267738,0.598837,4.084349,6.308098,6.240276,5.686975
75924,2022-04-30 08:15:00,4.60477,0.235862,0.116894,0.150143,4.056989,8.421783,4.334673,4.240579,4.365786,...,2.268957,4.282792,6.558778,2.173028,4.266077,0.598837,4.067183,6.308098,6.240276,5.686975
75925,2022-04-30 08:30:00,4.60477,0.235862,0.116894,0.150143,4.056989,8.421783,4.334673,4.222480,4.371881,...,2.213551,4.284465,6.568148,2.157584,4.264743,0.598837,4.345836,6.308098,6.240276,5.686975


In [56]:
#train/test seperation
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(df_main, test_size=0.2, shuffle=False)
validation_set, test_set = train_test_split(test_set, test_size=0.5, shuffle=False)

validation_set.reset_index(inplace=True)
test_set.reset_index(inplace=True)

print('# of train_set : %.0f. # of validation_set : %.0f. # of test_set : %.0f' %(train_set.shape[0],  validation_set.shape[0], test_set.shape[0]))

# of train_set : 60741. # of validation_set : 7593. # of test_set : 7593


In [57]:
class LTSF_Linear(torch.nn.Module):
    def __init__(self, window_size, forcast_size, individual, feature_size):
        super(LTSF_Linear, self).__init__()
        self.window_size = window_size
        self.forcast_size = forcast_size
        self.individual = individual
        self.channels = feature_size
        if self.individual:
            self.Linear = torch.nn.ModuleList()
            for i in range(self.channels):
                self.Linear.append(torch.nn.Linear(self.window_size, self.forcast_size))
        else:
            self.Linear = torch.nn.Linear(self.window_size, self.forcast_size)

    def forward(self, x):
        if self.individual:
            output = torch.zeros([x.size(0),self.pred_len,x.size(2)],dtype=x.dtype).to(x.device)
            for i in range(self.channels):
                output[:,:,i] = self.Linear[i](x[:,:,i])
            x = output
        else:
            x = self.Linear(x.permute(0,2,1)).permute(0,2,1)
        return x

In [58]:
def time_slide_df(df, window_size, forcast_size, date, target):
    df_ = df.copy()
    data_list = []
    dap_list = []
    date_list = []
    for idx in range(0, df_.shape[0]-window_size-forcast_size+1):
        x = df_.loc[idx:idx+window_size-1, target].values.reshape(window_size, 1)
        y = df_.loc[idx+window_size:idx+window_size+forcast_size-1, target].values
        date_ = df_.loc[idx+window_size:idx+window_size+forcast_size-1, date].values
        data_list.append(x)
        dap_list.append(y)
        date_list.append(date_)
    return np.array(data_list, dtype='float32'), np.array(dap_list, dtype='float32'), np.array(date_list)

class Data():
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __len__(self):
        return len(self.Y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

In [72]:
### Univariable ###
### 데이터 셋 생성 ###
window_size = 420
forcast_size = 1
batch_size = 64
targets = 'SATURATOR_ML_SUPPLY_F_PV.Value'
date = 'TimeStamp'

train_x, train_y, train_date = time_slide_df(train_set, window_size, forcast_size, date, targets)
val_x, val_y, val_date = time_slide_df(validation_set, window_size, forcast_size, date, targets)
test_x, test_y, test_date = time_slide_df(test_set, window_size, forcast_size, date, targets)

In [73]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [74]:
from torch.utils.data import DataLoader
train_ds = Data(train_x, train_y)
valid_ds = Data(val_x, val_y)
test_ds = Data(test_x, test_y)

train_dl = DataLoader(train_ds, batch_size = batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size = val_x.shape[0], shuffle=False)
test_dl  = DataLoader(test_ds,  batch_size = test_x.shape[0], shuffle=False)

In [75]:
### 모델 학습 ###
from tqdm.auto import tqdm

train_loss_list = []
valid_loss_list = []
test_loss_list = []
epoch = 200
lr = 0.001
model = LTSF_Linear(window_size=window_size,
                            forcast_size=forcast_size,
                            individual=False,
                            feature_size=1,
                            ).to(device)
                            
criterion = torch.nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
max_loss = 10**10

for epoch in tqdm(range(1, epoch+1)):
    loss_list = []
    model.train()
    for batch_idx, (data, target) in enumerate(train_dl):
        optimizer.zero_grad()
        output = model(data.to(device))
        loss = criterion(output.to(device), target.to(device).unsqueeze(-1))
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())    
    train_loss_list.append(np.mean(loss_list))

    model.eval()
    with torch.no_grad():
        for data, target in valid_dl:
            output = model(data.to(device))
            valid_loss = criterion(output.to(device), target.to(device).unsqueeze(-1))
            valid_loss_list.append(valid_loss)
        
        for data, target in test_dl:
            output = model(data.to(device))
            test_loss = criterion(output, target.to(device).unsqueeze(-1))
            test_loss_list.append(test_loss)

    if valid_loss < max_loss:
        torch.save(model, 'Linear_model.pth')
        max_loss = valid_loss
        #print("valid_loss={:.3f}, test_los{:.3f}, Model Save".format(valid_loss, test_loss))
        best_epoch = epoch
        best_train_loss = np.mean(loss_list)
        best_valid_loss = np.mean(valid_loss.item())
        best_test_loss = np.mean(test_loss.item())

    print("epoch = {}, train_loss(RMSE) : {:.3f}, valid_loss(RMSE) : {:.3f}, test_loss(RMSE) : {:.3f}".format(epoch,
                                                                                                              torch.sqrt(torch.mean(torch.Tensor(loss_list))),
                                                                                                              torch.sqrt(torch.Tensor(valid_loss)),
                                                                                                              torch.sqrt(torch.Tensor(test_loss))
                                                                                                              )
                                                                                                            )

  0%|          | 0/200 [00:00<?, ?it/s]

epoch = 1, train_loss(RMSE) : 0.470, valid_loss(RMSE) : 0.126, test_loss(RMSE) : 0.132
epoch = 2, train_loss(RMSE) : 0.116, valid_loss(RMSE) : 0.120, test_loss(RMSE) : 0.127
epoch = 3, train_loss(RMSE) : 0.116, valid_loss(RMSE) : 0.119, test_loss(RMSE) : 0.126
epoch = 4, train_loss(RMSE) : 0.115, valid_loss(RMSE) : 0.107, test_loss(RMSE) : 0.113
epoch = 5, train_loss(RMSE) : 0.113, valid_loss(RMSE) : 0.117, test_loss(RMSE) : 0.121
epoch = 6, train_loss(RMSE) : 0.114, valid_loss(RMSE) : 0.098, test_loss(RMSE) : 0.102
epoch = 7, train_loss(RMSE) : 0.109, valid_loss(RMSE) : 0.139, test_loss(RMSE) : 0.141
epoch = 8, train_loss(RMSE) : 0.105, valid_loss(RMSE) : 0.130, test_loss(RMSE) : 0.133
epoch = 9, train_loss(RMSE) : 0.106, valid_loss(RMSE) : 0.087, test_loss(RMSE) : 0.090
epoch = 10, train_loss(RMSE) : 0.100, valid_loss(RMSE) : 0.131, test_loss(RMSE) : 0.133
epoch = 11, train_loss(RMSE) : 0.098, valid_loss(RMSE) : 0.083, test_loss(RMSE) : 0.085
epoch = 12, train_loss(RMSE) : 0.101, val